In [21]:
from nltk.stem import SnowballStemmer 
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import math
import string 
string.punctuation
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# import sys
# sys.stdout.reconfigure(encoding='utf-8')

[nltk_data] Downloading package stopwords to /home/bkoch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bkoch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: 'OutStream' object has no attribute 'reconfigure'

In [30]:
# data_pth='data'
corpus_pth='data/corpus'
input_pth='data/corpus/goose/Chinese goose.txt'

In [3]:
import os
txt_file_pths=[]
for root, dirs, files in os.walk(corpus_pth, topdown=False):
   for name in files:
      txt_file_pths.append(os.path.join(root, name))


In [4]:
texts=[open(pth,'r').read() for pth in txt_file_pths]

In [5]:
tokenizer=word_tokenize
stemmer=SnowballStemmer(language='english', ignore_stopwords=False)
punctuation='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [41]:
def preprocessor(txt):
    tokens=[stemmer.stem(token.lower()) for token in tokenizer(txt) if token.isalnum()]
    return tokens

preprocessor('hello ( ) my friend running. ')

['hello', 'my', 'friend', 'run']

In [7]:
tok_corpus=[preprocessor(text) for text in texts]

In [8]:
def tfidf(tok_doc, tok_corpus):
    doc_length=len(tok_doc)
    corpus_length=len(tok_corpus)

    #tf
    term_count=Counter(tok_doc)
    term_frequency={key:value/doc_length for key, value in term_count.items()}
    
    #idf
    inverse_doc_frequency={}
    for key in tok_doc:
        doc_frequency=0
        for doc in tok_corpus:
            if key in doc:
                doc_frequency+=1
        inverse_doc_frequency[key]=math.log(corpus_length/doc_frequency)
    
    #tfidf
    tfidf={}
    for key in tok_doc:
        tfidf[key]=term_frequency[key] * inverse_doc_frequency[key]
        
    return tfidf
    


In [9]:
# Unit test
text='Joseph James Rogan (born August 11, 1967) is an American comedian, podcast host, and mixed martial arts color commentator. He has also worked as a television host and an actor.'
tok_text=preprocessor(text)
tfidf(tok_text, tok_corpus)

{'joseph': 0.056622301538734765,
 'jame': 0.04874267560326042,
 'rogan': 0.1468906415754751,
 'born': 0.04095551389721024,
 'august': 0.04095551389721024,
 '11': 0.04874267560326042,
 '1967': 0.07757592351948057,
 'is': 0.00040900308639381334,
 'an': 0.007750653741057468,
 'american': 0.02744001029360477,
 'comedian': 0.11027023195320478,
 'podcast': 0.11027023195320478,
 'host': 0.10108983262453923,
 'and': 0.0016461741726914276,
 'mix': 0.0382874236414257,
 'martial': 0.12378573555681026,
 'art': 0.04095551389721024,
 'color': 0.03959478074653508,
 'comment': 0.05245019677360123,
 'he': 0.019269261692505268,
 'has': 0.008261205463486042,
 'also': 0.007235483507927586,
 'work': 0.015829787151330914,
 'as': 0.0020971275136856394,
 'a': 0.0,
 'televis': 0.050544916312269615,
 'actor': 0.09324271116100509}

In [10]:
tok_text=preprocessor(open(input_pth,'r').read())

In [11]:
result=tfidf(tok_text, tok_corpus)

In [12]:
result=[(k,v) for k, v in sorted(result.items(), key=lambda item: item[1], reverse=True)][:10]
result

[('chines', 0.07214946602648957),
 ('goos', 0.06933343045559359),
 ('breed', 0.06763759193657246),
 ('gees', 0.061494611463241304),
 ('knob', 0.05684038877608635),
 ('swan', 0.0428155306351554),
 ('lay', 0.03381879596828623),
 ('femal', 0.026000036420847597),
 ('wild', 0.025012201913639877),
 ('descend', 0.023747731689636908)]

In [13]:
', '.join([k for k,v in result])

'chines, goos, breed, gees, knob, swan, lay, femal, wild, descend'

In [43]:
def task1(corpus_pth, input_pth):
    
    #find all txt files
    txt_file_pths=[]
    for root, dirs, files in os.walk(corpus_pth, topdown=False):
       for name in files:
          txt_file_pths.append(os.path.join(root, name))
    
    #read them
    texts=[open(pth,'r').read() for pth in txt_file_pths]
    
    #preprocess them
    tok_corpus=[preprocessor(text) for text in texts]
    
    #read input doc
    tok_text=preprocessor(open(input_pth,'r').read())
    
    #calculate tfidf for each term
    tfidf_scores=tfidf(tok_text,tok_corpus)
    
    # extract top 10
    top_10=[(k,v) for k, v in sorted(tfidf_scores.items(), key=lambda item: item[1], reverse=True)][:10]
    
    #print top 10
    print(', '.join([k for k,v in top_10]))
    
    return tfidf_scores
    

    

In [44]:
tfidf_scores=task1(corpus_pth,input_pth)

chines, goos, breed, gees, knob, swan, lay, femal, wild, descend


Extract the summary of a document using a technique based on TF-IDF. For each sentence, the sentence relevance score is defined as the sum of the TF-IDF scores of the most important 10 words in the sentence. The summary is then obtained by taking the 5 sentences with the greatest relevance scores, in the same order in which they appear in the original document. (If a sentence has fewer than 10 words, its sentence score is simply the sum of TF-IDF scores of all its words. If the document consists of 5 or fewer sentences, the summary is simply the entire document.)

In [46]:
def task2(text,tok_corpus):
    sentences=sent_tokenize(text)
    print(len(sentences))
    if len(sentences)<6:
        return text # da li treba spojit s razmakom ili samo text vratit 
    
    sent_scores=[]
    tok_text=preprocessor(text)
    tfidf_score=tfidf(tok_text, tok_corpus)
    
    for sent in sentences:
        tok_sent=preprocessor(sent)
        tfidfs=[tfidf_score[tok] for tok in tok_sent]
        tfidfs_top_10=sorted(tfidfs,reverse=True)[:10]
        sent_score=sum(tfidfs_top_10)

        sent_scores.append((sent,sent_score))
    
    sentences_result=[]
    for sent,val in sorted(sent_scores, key=lambda item: item[1], reverse=True)[:5]:
        sentences_result.append(sent)
        
    sentences_sorted=[]
    for sent in sentences:
        if sent in sentences_result:
            sentences_sorted.append(sent)
        
    print('\n'.join(sentences_sorted))
    
task2(open(input_pth,'r').read(), tok_corpus)   

9
The Chinese goose is a breed of domesticated goose descended from the wild swan goose.
Chinese geese differ from the wild birds in much larger size (up to 5–10 kg in males, 4–9 kg in females), and in having an often strongly developed basal knob on the upper side of the bill.
Chinese geese are a close cousin of the African goose, a heavier breed also descended from the swan goose.
While many domestic Chinese geese have a similar body type to other breeds, the breed standards as defined in the American Poultry Association's Standard of Perfection and other sources call for a slimmer, taller fowl.
A female Chinese goose can lay 50–60 eggs over the course of the breeding season (February to June), although there are reports of Chinese geese laying up to 100 eggs during that time.


The Chinese goose is a breed of domesticated goose descended from the wild swan goose.

Chinese geese differ from the wild birds in much larger size (up to 5–10 kg in males, 4–9 kg in females), and in having an often strongly developed basal knob on the upper side of the bill.

Chinese geese are a close cousin of the African goose, a heavier breed also descended from the swan goose.

While many domestic Chinese geese have a similar body type to other breeds, the breed standards as defined in the American Poultry Association's Standard of Perfection and other sources call for a slimmer, taller fowl.

A female Chinese goose can lay 50–60 eggs over the course of the breeding season (February to June), although there are reports of Chinese geese laying up to 100 eggs during that time.


In [19]:
?sent_tokenize